In [3]:
import pandas as pd
import numpy as np
import cvxpy as cp

## Optimization Model

In [11]:
def NPM(r, his_r, x_tilde, theta, c):
    '''
    param r [ndarray]: vector of price at period t
    param his_r [ndarray]: 2d vector of prices for the past W periods
    param x_tilde [ndarray]: investment weights at the end of period t-1
    param theta [int]: acceptable risk level
    param c [float]: transacion cost \in (0, 1)

    '''
    M = len(r) # Number of assets in the portfolio
    W = len(his_r) # Number of recents prices used to measure risk
    
    # Define variables
    u = cp.Variable(M)
    v = cp.Variable(M)
    d = cp.Variable(W)
    g = cp.Variable(W)
    
    # Define Objective
    objective = cp.Maximize(r @ x_tilde + (x_tilde - c/2) @ u) - (x_tilde + c/2) @ v)
    
    # Define constraints
    constraints = [cp.sum(d + g) <= W * theta,
                    cp.sum(u - v) == 0,
                    u - v + x_tilde >= 0,
                    u - v + x_tilde <= 1,
                    u >= 0, v >= 0, d >= 0, g >= 0]
    for j in range(M):
        constraints += [(his_r[j] - r) @ u + (his_r[j] - r) @ v - d[j] <= (r - his_r[j]) @ x_tilde,
                        (r - his_r[j]) @ u + (r - his_r[j]) @ v - d[j] <= (his_r[j] - r) @ x_tilde]
        
    # Solve the optimization problem
    problem = cp.Problem(objective, constraints)
    problem.solve()
    
    return problem, u.value, v.value, d.value, g.value